In [1]:
!pip install tensorflow_text
!pip install transformers

     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 462 kB 57.4 MB/s 
     |████████████████████████████████| 3.5 MB 5.1 MB/s 
     |████████████████████████████████| 6.8 MB 47.3 MB/s 
     |████████████████████████████████| 895 kB 66.6 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 596 kB 71.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/Development/
  inflating: data/Development/dev_definitions.txt  
  inflating: data/Development/dev_examples.txt  
  inflating: data/Development/dev_hypernyms.txt  
  inflating: data/Development/dev_labels.txt  
  inflating: data/README.txt         
   creating: data/Test/
  inflating: data/Test/test_definitions.txt  
  inflating: data/Test/test_examples.txt  
  inflating: data/Test/test_hypernyms.txt  
   creating: data/Training/
  inflating: data/Training/train_definitions.txt  
  inflating: data/Training/train_examples.txt  
  inflating: data/Training/train_hypernyms.txt  
  inflating: data/Training/train_labels.txt  


In [3]:
import re
import os
import errno
import pandas as pd

!rm -rf processed_data

try:
    os.makedirs("processed_data")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

def decontracted(phrase):

    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can't", "can not", phrase)
    phrase = re.sub(r"gonna", "going to", phrase)
    phrase = re.sub(r"wanna", "want to", phrase)

    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\.+", " ", phrase)
    phrase = re.sub(r"[^A-Za-z$]", " ", phrase)
    phrase = re.sub(r" +", " ", phrase)

    return phrase


def get_df(pref, testing=False):

    dataset = []
    with open(pref + "_" + "examples.txt") as examples:
        for example in examples:
            example = example[:-1].lower().split("\t")
            if example[0] != decontracted(example[0]):
                text = re.sub(example[0], "\$", example[2])
                text = decontracted(text)
                text = re.sub(r"\$", example[0], text)
                example[2] = text
            else:
                example[2] = decontracted(example[2])
            dataset.append(example)

    with open(pref + "_" + "hypernyms.txt") as hypernyms:
        for index, line in enumerate(hypernyms):
            line = line[:-1].lower().split("\t")
            line = " ; ".join(line)
            line = re.sub(r'_', ' ', line)
            dataset[index].append(line)

    with open(pref + "_" + "definitions.txt") as definitions:
        for index, line in enumerate(definitions):
            line = line[:-1].lower().split(";")
            for i in range(len(line)):
                line[i] = line[i].strip()
                if dataset[index][0] != decontracted(dataset[index][0]):
                    text = re.sub(dataset[index][0], "\$", line[i])
                    text = decontracted(text)
                    text = re.sub(r"\$", dataset[index][0], text)
                    line[i] = text
                else:
                    line[i] = decontracted(line[i])
            line = " ; ".join(line)
            dataset[index].append(line)

    cols = ["target", "position", "sentence", "hypernym", "definition"]

    if not testing:
        cols.append("label")
        with open(pref + "_" + "labels.txt") as labels:
            for index, line in enumerate(labels):
                line = line[:-1]
                dataset[index].append(line)

    df = pd.DataFrame(dataset, columns=cols)

    return df


dev_df = get_df("data/Development/dev")
train_df = get_df("data/Training/train")
test_df = get_df("data/Test/test", True)

dev_df.to_csv("processed_data/dev.csv", index=None)
train_df.to_csv("processed_data/train.csv", index=None)
test_df.to_csv("processed_data/test.csv", index=None)

In [4]:
import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import pandas as pd

from keras import backend as K

import transformers
from transformers import BertTokenizer
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [5]:
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", trainable = True)

In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

transformers.logging.set_verbosity_error()

def extract_data(filename):

    df = pd.read_csv(filename)
    X = {'input_type_ids':[],'input_word_ids':[],'input_mask':[], 'sep1': [], 'sep2': []}
    Y = []

    for _, row in df.iterrows():
        word = row['target']
        sentence = row['sentence']
        hypernyms = row['hypernym']
        definitions = row['definition']
        if row['label'] == 'T':
            label = 1.0
        else:
            label = 0.0

        desc = [definitions]
        if isinstance(hypernyms, str):
          desc.append(hypernyms)

        desc = ' ; '.join(desc)

        x = tokenizer(sentence, desc, max_length = 128, padding='max_length', truncation = True)

        x['sep1'] = x['input_ids'].index(102)
        x['sep2'] = x['input_ids'].index(102, x['sep1'] + 1)

        X['input_type_ids'].append(x['token_type_ids'])
        X['input_word_ids'].append(x['input_ids'])
        X['input_mask'].append(x['attention_mask'])
        X['sep1'].append(x['sep1'])
        X['sep2'].append(x['sep2'])

        Y.append(label)     

    X['input_type_ids'] = np.array(X['input_type_ids'], dtype=np.float32)
    X['input_word_ids'] = np.array(X['input_word_ids'], dtype=np.float32)
    X['input_mask'] = np.array(X['input_mask'], dtype=np.float32)

    X['sep1'] = np.array(X['sep1'], dtype=np.float32)
    X['sep2'] = np.array(X['sep2'], dtype=np.float32)

    Y = np.array(Y)

    return X, Y

X,Y = extract_data('processed_data/train.csv')
Xval, Yval = extract_data('processed_data/dev.csv')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
input_type_ids = tf.keras.layers.Input(shape=(128,),dtype=tf.int32, name="input_type_ids")
input_word_ids = tf.keras.layers.Input(shape=(128,),dtype=tf.int32, name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(128,),dtype=tf.int32, name="input_mask")

input_sep1 = tf.keras.layers.Input(shape=(1,),dtype=tf.int32, name="input_sep1")
input_sep2 = tf.keras.layers.Input(shape=(1,),dtype=tf.int32, name="input_sep2")

pooled_output, sequence_output = bert_encoder([input_word_ids, input_mask, input_type_ids])

def slice(tensors):
    sep1 = tensors[1]
    sep2 = tensors[2]
    tensor = tensors[0]
    mod_tensor1 = []
    mod_tensor2 = []
    for i in range(16):
        mod_tensor1.append(K.mean(tensor[i, : sep1[i][0], :], axis = 0))
        mod_tensor2.append(K.mean(tensor[i, sep1[i][0]: sep2[i][0], :], axis = 0))
    mod_tensor1 = tf.stack(mod_tensor1)
    mod_tensor2 = tf.stack(mod_tensor2)
    return mod_tensor1, mod_tensor2

seq1, seq2 = tf.keras.layers.Lambda(slice)([sequence_output, input_sep1, input_sep2])

x = tf.keras.layers.Concatenate()([seq1, seq2, pooled_output])
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model = tf.keras.models.Model(
      inputs=[
        input_word_ids,
        input_mask,
        input_type_ids,
        input_sep1,
        input_sep2], 
      outputs=x)

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 128)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 128, 768)]                'input_mask[0][0]',         

In [8]:
model.fit([X['input_word_ids'], X['input_mask'], X['input_type_ids'], X['sep1'], X['sep2']], Y, 
          epochs = 20, batch_size = 16, validation_batch_size = 16,
          validation_data = ([Xval['input_word_ids'], Xval['input_mask'], Xval['input_type_ids'], Xval['sep1'], Xval['sep2']], Yval)
          )

Epoch 1/20
133/133 [==============================] - 36s 179ms/step - loss: 0.6420 - accuracy: 0.6443 - val_loss: 0.6496 - val_accuracy: 0.6536
Epoch 2/20
133/133 [==============================] - 23s 176ms/step - loss: 0.5947 - accuracy: 0.6856 - val_loss: 0.6629 - val_accuracy: 0.6224
Epoch 3/20
133/133 [==============================] - 25s 190ms/step - loss: 0.5693 - accuracy: 0.6992 - val_loss: 0.6309 - val_accuracy: 0.6589
Epoch 4/20
133/133 [==============================] - 25s 187ms/step - loss: 0.5466 - accuracy: 0.7110 - val_loss: 0.6327 - val_accuracy: 0.6484
Epoch 5/20
133/133 [==============================] - 24s 179ms/step - loss: 0.5305 - accuracy: 0.7317 - val_loss: 0.6492 - val_accuracy: 0.6562
Epoch 6/20
133/133 [==============================] - 25s 188ms/step - loss: 0.5330 - accuracy: 0.7378 - val_loss: 0.6331 - val_accuracy: 0.6484
Epoch 7/20
133/133 [==============================] - 25s 188ms/step - loss: 0.5218 - accuracy: 0.7378 - val_loss: 0.6193 - val_ac